In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import pandas as pd
# The dataset changed a little in _r1: "acctime" used to be all 0:00:00. 
# In the revised csv `acctime` is converted to integer. 
# There's probably a number of entries that fall beyond 2359 (and possibly a few negative ones as well), so we'll have to clean those. 
df0 = pd.read_csv("../input/toronto-collisions-dataset-r1/involved2020_forinsight_r1.csv")

In [ ]:
# df0 is a mix of two datasets: "Collisions-Events" and "Collisions-Involved"
df0.head(10)

In [ ]:
df0.shape

In [ ]:
df0.columns

In [ ]:
# This shows the number of collisions itself (NOT the number of people involved in the collisions)
df0['collision_no'].nunique()

In [ ]:
# Create df1 from df0
df1 = df0.copy()

# Add new column 'acc_date'
df1['acc_date'] = pd.to_datetime(df0['accdate'], errors = 'raise')
df1['year'] = df1['acc_date'].dt.year
df1['month_name'] = df1['acc_date'].dt.month_name()
df1['day_name'] = df1['acc_date'].dt.day_name()

# Drop columns 'accdate' and 'accyear'
df1.drop(['accdate', 'accyear'], axis=1, inplace=True)

In [ ]:
df1['acc_date'].dt.year.unique() 

In [ ]:
df1['involved_injury_class'].unique()

In [ ]:
print('{:,}'.format(df1['involved_injury_class'].isna().sum()), 
      'data points in "involved_injury_class" are nan, which will be removed.')
df1.dropna(subset=['involved_injury_class'], inplace=True)

In [ ]:
df1['involved_class'].unique()

In [ ]:
print('{:,}'.format(df1['involved_class'].isna().sum()), 
      'data points in "involved_class" are nan, which will be removed.')
df1.dropna(subset=['involved_class'], inplace=True)

In [ ]:
print('{:,}'.format(df1['longitude'].isna().sum()), 
      'data points in "longitude" are nan, which will be removed.')
df1.dropna(subset=['longitude'], inplace=True)

In [ ]:
df1['latitude'].isna().sum()

In [ ]:
# Drop points whose lon-lat are not in Toronto (spatial outlier)
# Toronto lon-lat ranges:
# longitude: (-79.65 , -79.08)
# latitude: (43.57 , 43.85)

i = df1[(df1['longitude']<-79.65) | (df1['longitude']>-79.08) | (df1['latitude']<43.57) | (df1['latitude']>43.85)].index
df1.drop(i, inplace=True)

print('There were {:,} spatial outliers which are removed now.'.format(i.size))

In [ ]:
# Add new columns to df1
df1['# involved'] = int(1)

df1['# injured'] = int(0)
df1.loc[df1['involved_injury_class'].isin(['MINIMAL', 'MINOR', 'MAJOR']), ['# injured']] = int(1) 

df1['# fatalities'] = int(0)
df1.loc[df1['involved_injury_class']=='FATAL', ['# fatalities']] = int(1) 

df1['# KSI'] = int(0)
df1.loc[df1['involved_injury_class'].isin(['MAJOR', 'FATAL']), ['# KSI']] = int(1) 

In [ ]:
# Create dfE: the dataframe of "Collisions-Events" from df1

# We don't include the following columns which are related to "Collisions-Involved" except 'involved_class' and involved_injury_class': 
#                                     'traffic_control', 'vehicle_no', 'vehicle_class', 'initial_dir', 'impact_type', 'safety_equip_used', 
#                                     'driver_action','driver_condition', 
#                                     'pedestrian_action', 'pedestrian_condition', 'pedestrian_collision_type', 
#                                     'cyclist_action', 'cyclist_condition', 'cyclist_collision_type', 
#                                     'manoeuver', 'posted_speed', 'actual_speed'.

# Also we don't iclude the following columns which are not going to be useful for our analysis although they belong to "Collisions-Events":
#                                     'stname1', 'streetype1', 'dir1', 'stname2', 'streetype2', 'dir2', 'stname3', 'streetype3', 'dir3'.

dfE = df1.groupby(['collision_no', 'acc_date', 'year', 'month_name', 'day_name', 'acctime', 'longitude','latitude',
                   'road_class', 'road_surface_cond', 'visibility', 'light']).\
agg({'involved_class': ', '.join,
     'involved_injury_class': ', '.join,
    '# involved':'sum',
    '# injured': 'sum',
    '# fatalities': 'sum',
    '# KSI': 'sum'})

dfE.reset_index(inplace=True)
dfE.head()

When preparing to share maps or layers on the Web, it is recommended to reproject your source data to the **Web Mercator** coordinate system. Doing so will ensure that your map data is located correctly and aligns properly with other services such as popular content providers Microsoft® Bing™ Maps, Google Maps™, and ESRI® ArcGISSM Online, which have standardized their services on the Web Mercator coordinate system.

In [ ]:
import datashader as ds
from datashader.utils import lnglat_to_meters

# Project longitude and latitude onto web mercator plane
dfE.loc[:, 'X'], dfE.loc[:, 'Y'] = lnglat_to_meters(dfE['longitude'], dfE['latitude'])

In [ ]:
# Change the order of columns
cols = dfE.columns.tolist() 
new_cols = cols[0:8] + [cols[18]] + [cols[19]] + cols[8:18]
dfE = dfE.reindex(columns=new_cols)
dfE.head()

In [ ]:
# dfE.to_csv('dfE.csv')
# It will be exported for the other notebooks: 'Maps', 'HeatMap', 'Graphs'

## After July 2019 minor and non-injury (property damage) collisions are not updated by Collision Reporting Centre (CRC), but we assume that the fatalities and major injuries are updated by Toronto Police Services (TPS) 

---

In [ ]:
# Create the dataframe of neighborhoods (neighb) in Toronto. It's a shape file(.shp):
import geopandas as gpd
neighb = gpd.read_file("../input/neighbourhoods-toronto-2/Neighbourhoods.shp", encoding="utf-8")
neighb

In [ ]:
# It sounds like both 'FIELD_7' and 'FIELD_8' show the name of the neighborhoos.
# So check if there is any differences between them: 
all(neighb['FIELD_7']==neighb['FIELD_8'])

# 'FIELD_7' is actually neighborhood NAME and 'FIELD_8' is DESC.

In [ ]:
# Insight is located in the 'Church-Yonge Corridor (75)' neighborhood
INSIGHT = neighb.loc[neighb['FIELD_7']=='Church-Yonge Corridor (75)']
INSIGHT

In [ ]:
INSIGHT.reset_index(drop=True, inplace=True)
fig, ax = plt.subplots(figsize=(15,15))
neighb.plot(ax=ax, facecolor='gray')
INSIGHT.plot(ax=ax, facecolor='red')
plt.tight_layout()

In [ ]:
# The next step is to get the data in the right format. 
# The way we do this is by turning our regular Pandas DataFrame into a geo-DataFrame, which will require us to specify as parameters:
#    the original DataFrame, 
#    our coordinate reference system (CRS), and 
#    the geometry of our new DataFrame.

# In order to format our geometry appropriately, we will need to convert the longitude and latitude into Points (imported from shapely).

from shapely.geometry import Point, Polygon

crs = {'init': 'epsg:4326'}

geometry = [Point(xy) for xy in zip(dfE['longitude'], dfE['latitude'])]   # Make sure to always specify the “Longitude” column before the “Latitude” column!

geo_dfE = gpd.GeoDataFrame(dfE, # specify our data
                           crs=crs, # specify our coordinate reference system
                           geometry=geometry) # specify the geometry list we created
geo_dfE.head()

In [ ]:
fig, ax = plt.subplots(figsize=(15,15))
neighb.plot(ax=ax, alpha=1, facecolor='gray')
geo_dfE[geo_dfE['acc_date'].dt.year==2018].plot(ax=ax, markersize=1, color='blue', marker='o', label='Number of collisions in 2018')
plt.legend(prop={'size': 15})

Computationally, detecting if a point is inside a polygon is most commonly done using a specific formula called Ray Casting algorithm. Luckily, we do not need to create such a function ourselves for conducting the Point in Polygon (PIP) query. Instead, we can take advantage of Shapely’s binary predicates that can evaluate the topolocical relationships between geographical objects, such as the PIP as we’re interested here.

There are basically two ways of conducting PIP in Shapely:

    - using a function called .within() that checks if a point is within a polygon
    - using a function called .contains() that checks if a polygon contains a point

Notice: even though we are talking here about Point in Polygon operation, it is also possible to check if a LineString or Polygon is inside another Polygon.

In [ ]:
# Check which collision points are in INSIGHT neighborhood
pip_mask = geo_dfE.within(INSIGHT.loc[0, 'geometry'])
print(pip_mask)

In [ ]:
# geo_dfE['neighborhood'] = " "

# # start = time.time()

# for i in range(len(geo_dfE)):
#     for j in range(len(neighb)):
#         if geo_dfE.loc[i, 'geometry'].within(neighb.loc[j, 'geometry']):
#             geo_dfE.loc[i, 'neighborhood'] = neighb.loc[j, 'FIELD_7']
#             break

# # elapsed = (time.time() - start)
# # from datetime import timedelta
# # str(timedelta(seconds=elapsed))

# geo_dfE.head

In [ ]:
pip_data = geo_dfE.loc[pip_mask]

In [ ]:
INSIGHT.reset_index(drop=True, inplace=True)
fig, ax = plt.subplots(figsize=(15,15))
neighb.plot(ax=ax, alpha=0.8, facecolor='gray')
INSIGHT.plot(ax=ax, facecolor='red')
pip_data.plot(ax=ax, color='blue', markersize=0.001)     # plot also the points on top of the map
plt.tight_layout()

In [ ]:
# Create the geo dataframe of INSIGHT ('Church-Yonge Corridor (75)')
geo_dfE_I = pip_data
geo_dfE_I

In [ ]:
# It will be used for the notebook 'Regression'
# geo_dfE_I.to_csv('geo_dfE_I.csv')